In [7]:
cd ..

/home/wbaik/Code/data_science/temp_folder


In [8]:
import plotly

In [9]:
plotly.__version__

'2.5.0'

In [10]:
from postgres_utils import PostgresPandas
from reprocess import merge_dataframes

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning:

The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.



In [11]:
ppd = PostgresPandas()

intersted = ['shak', 'cmg', 'gs', 'tsla', 'amzn']

In [12]:
ret = ppd.get_tables(*intersted)

In [13]:
ret.keys()

dict_keys(['shak', 'cmg', 'gs', 'tsla', 'amzn'])

In [14]:
neo_table = merge_dataframes(ret)

In [15]:
neo_table.drop('date', axis=1).head()

,close_shak,close_cmg,close_gs,close_tsla,close_amzn
0,45.90,709.84,165.99,203.60,354.53
1,43.99,712.55,168.97,210.94,364.47
2,44.98,726.63,173.53,218.36,363.55
3,41.32,676.00,172.14,218.55,364.75
4,42.64,670.91,174.04,220.99,373.89


In [16]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py
from plotly.grid_objs import Grid, Column
from plotly.tools import FigureFactory as FF
import time

In [17]:
table = FF.create_table(neo_table.head(10))
iplot(table, filename='animations-preview')

/usr/local/lib/python3.6/dist-packages/plotly/tools.py:1491: UserWarning:

plotly.tools.FigureFactory.create_table is deprecated. Use plotly.figure_factory.create_table



In [18]:
init_notebook_mode(connected=True)

In [19]:
iplot(table, filename='animations-preview')


In [20]:
dates_from_col = neo_table['date']

In [21]:
dates_from_col_by_year = dates_from_col.apply(lambda x: x[:4])
dates_from_col_by_year[:3]

0    2015
1    2015
2    2015
Name: date, dtype: object

In [22]:
import plotly.graph_objs as go
import plotly.figure_factory as ff

In [23]:
traces = []
for i in ret:
    df = ret[i]
    trace = go.Ohlc(x=df['date'],
                    open=df.open,
                    high=df.high,
                    low=df.low,
                    close=df.close)
                    
    traces.append(trace)
py.iplot(traces)

In [25]:
neo_table.drop('date', axis=1).pct_change()[:5]

,close_shak,close_cmg,close_gs,close_tsla,close_amzn
0,NaN,NaN,NaN,NaN,NaN
1,-0.041612,0.003818,0.017953,0.036051,0.028037
2,0.022505,0.019760,0.026987,0.035176,-0.002524
3,-0.081369,-0.069678,-0.008010,0.000870,0.003301
4,0.031946,-0.007530,0.011038,0.011164,0.025058


In [26]:
shaks = neo_table['close_shak'].pct_change()

In [27]:
cmg = neo_table['close_cmg'].pct_change()

In [31]:
figs = []
fig = ff.create_2d_density(
    shaks, cmg
)
figs.append(fig)
figs.append(fig)

for i in figs:
    py.iplot(i, subplots=True)

TypeError: iplot() got an unexpected keyword argument 'subplots'

In [29]:
close = ret['cmg']['close'].pct_change()*100
volume = ret['cmg']['volume']

fig = ff.create_2d_density(
    close, volume
)

py.iplot(fig)

In [34]:
import numpy as np

In [160]:
neo_table.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            693, 694, 695, 696, 697, 698, 699, 700, 701, 702],
           dtype='int64', length=703)

In [242]:
here = neo_table.pivot_table(index=['year', 'month', 'date'])
here = here.pct_change()

In [243]:
here[:10]

close_amzn  close_cmg  close_gs  close_shak  close_tsla
year month date                                                               
2015 01    2015-01-30         NaN        NaN       NaN         NaN         NaN
     02    2015-02-02    0.028037   0.003818  0.017953   -0.041612    0.036051
           2015-02-03   -0.002524   0.019760  0.026987    0.022505    0.035176
           2015-02-04    0.003301  -0.069678 -0.008010   -0.081369    0.000870
           2015-02-05    0.025058  -0.007530  0.011038    0.031946    0.011164
           2015-02-06    0.001043  -0.016381  0.014767   -0.015947   -0.016426
           2015-02-09   -0.009939  -0.018048 -0.006568   -0.010963    0.000552
           2015-02-10    0.006558   0.026265  0.012767    0.000000   -0.005472
           2015-02-11    0.005764   0.006932  0.016715    0.000000   -0.016136
           2015-02-12    0.005411   0.000971  0.011458   -0.020482   -0.046617

In [244]:
months = here.groupby('month', axis=0).all().index

In [245]:
months

Index(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], dtype='object', name='month')

In [246]:
here.groupby('month', axis=0).get_group('02').close_amzn[:4]

year  month  date      
2015  02     2015-02-02    0.028037
             2015-02-03   -0.002524
             2015-02-04    0.003301
             2015-02-05    0.025058
Name: close_amzn, dtype: float64

In [251]:
data = [dict(
    visible = False,
    line = dict(color='00CED1', width=6),
    name = 'v = '+str(step),
    y = here.groupby('month', axis=0).get_group(month).close_amzn,
    x = here.groupby('month', axis=0).get_group(month).close_cmg,
        ) for month in months]

data[10]['visible'] = True

In [257]:
data = [go.Scatter(
    y = here.groupby('month', axis=0).get_group(month).close_amzn,
    x = here.groupby('month', axis=0).get_group(month).close_cmg,
    mode='markers'
) for month in months]

In [260]:
steps = []
for i in range(len(data)):
    step = dict(
        method = 'restyle',
        args = ['visible', [False] * len(data)],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = 0,
    currentvalue = {"prefix": "Frequency: "},
    pad = {"t": 12},
    steps = steps
)]

layout = dict(sliders=sliders)

fig = dict(data=data, layout=layout)

py.iplot(fig, filename='Sine Wave Slider')

In [69]:
del ppd

In [74]:
ret['cmg'][:3]

,index,date,open,high,low,close,volume,openint
0,0,2006-01-25,22.00,22.00,22.00,22.0,0,0
1,1,2006-01-26,45.00,48.28,39.51,44.0,13219400,0
2,2,2006-01-27,44.88,44.90,41.75,42.2,1150800,0


In [84]:
neo_table.shape

(703, 6)

In [93]:
neo_table.ix[700:,:]

,date,close_shak,close_cmg,close_gs,close_tsla,close_amzn
700,2017-11-08,36.87,273.90,241.25,304.31,1132.88
701,2017-11-09,37.35,275.00,240.79,302.99,1129.13
702,2017-11-10,38.35,279.45,240.15,302.99,1125.35


In [108]:
neo_table['year'] = neo_table['date'].apply(lambda x: x.split('-')[0])

In [130]:
neo_table['month'] = neo_table['date'].apply(lambda x: x.split('-')[1])

In [131]:
neo_table.head()

,date,close_shak,close_cmg,close_gs,close_tsla,close_amzn,year,month
0,2015-01-30,45.90,709.84,165.99,203.60,354.53,2015,01
1,2015-02-02,43.99,712.55,168.97,210.94,364.47,2015,02
2,2015-02-03,44.98,726.63,173.53,218.36,363.55,2015,02
3,2015-02-04,41.32,676.00,172.14,218.55,364.75,2015,02
4,2015-02-05,42.64,670.91,174.04,220.99,373.89,2015,02


In [132]:
grouped_by_year_dict = {}

here = neo_table.pivot_table(index=['year', 'month', 'date'])
here[:10]

close_amzn  close_cmg  close_gs  close_shak  close_tsla
year month date                                                               
2015 01    2015-01-30      354.53     709.84    165.99       45.90      203.60
     02    2015-02-02      364.47     712.55    168.97       43.99      210.94
           2015-02-03      363.55     726.63    173.53       44.98      218.36
           2015-02-04      364.75     676.00    172.14       41.32      218.55
           2015-02-05      373.89     670.91    174.04       42.64      220.99
           2015-02-06      374.28     659.92    176.61       41.96      217.36
           2015-02-09      370.56     648.01    175.45       41.50      217.48
           2015-02-10      372.99     665.03    177.69       41.50      216.29
           2015-02-11      375.14     669.64    180.66       41.50      212.80
           2015-02-12      377.17     670.29    182.73       40.65      202.88

In [141]:
here.pct_change().groupby('month', axis=0).mean()

,close_amzn,close_cmg,close_gs,close_shak,close_tsla
month,,,,,
01,-0.000896,0.001667,-0.003739,-0.003448,-0.001387
02,0.000812,0.000982,0.002001,0.003068,0.000468
03,0.001541,-0.000444,-0.000478,-0.000040,0.003409
04,0.004770,-0.001397,0.001116,0.005534,0.005980
05,0.002958,0.000804,-0.000337,0.006068,0.001974
06,-0.000319,-0.003597,-0.000025,-0.006138,0.001421
07,0.004824,0.001260,0.001094,0.003295,0.000014
08,-0.000438,-0.002156,-0.000136,-0.006814,-0.000770
09,0.001127,0.000302,-0.001068,0.000353,-0.001153


In [142]:
here.pct_change().groupby('month', axis=0).var()

,close_amzn,close_cmg,close_gs,close_shak,close_tsla
month,,,,,
01,0.000773,0.000638,0.000341,0.000750,0.000471
02,0.000333,0.000409,0.000300,0.001369,0.001108
03,0.000149,0.000245,0.000178,0.000848,0.000461
04,0.000611,0.000324,0.000190,0.000857,0.000540
05,0.000147,0.000173,0.000192,0.001937,0.000438
06,0.000147,0.000292,0.000224,0.000636,0.000544
07,0.000284,0.000438,0.000129,0.001509,0.000630
08,0.000266,0.000188,0.000181,0.001228,0.000642
09,0.000220,0.000311,0.000224,0.000677,0.000399


In [156]:
mean_data.columns

Index(['close_amzn', 'close_cmg', 'close_gs', 'close_shak', 'close_tsla'], dtype='object')

In [159]:
mean_data = here.pct_change().groupby('month', axis=0).mean()

z = []
for i in mean_data:
    z.append(list(mean_data[i]))

data = [
    go.Heatmap(
        z = z,
        x = list(mean_data.index),
        y = list(mean_data.columns),
        colorscale='Viridis',
    )
]

fig = go.Figure(data=data)
py.iplot(fig)